This is a new file for doing the random forest algorith 

In [1]:
import numpy as np
from sklearn.externals import joblib
import pandas as pd

C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
# Our lovely TFIDF Matrix 
tfidf_matrix = joblib.load('../clustering/tfidf_matrix.pkl')

In [3]:
df = pd.read_csv('../data/opioid_tweets_label.csv').drop(columns = ["Unnamed: 0"])
y = df.label
df.label.value_counts()

0    42876
1       62
Name: label, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(tfidf_matrix ,df.label, test_size=0.2, random_state=0)

In [5]:
from sklearn.ensemble.forest import RandomForestClassifier #Be caresful with the  delcartion. 
classifier = RandomForestClassifier(n_estimators=1000, random_state=0)


C:\Users\JungleBook\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
%timeit classifier.fit(X_train, y_train) # In order to proceed we need something to classify our text with, in the exapmle used they
#use the use y which contains 0 or 1. 
#It is possible to create script and flag the tweets we manully identified to have illicit drugs. 
#What else would we compare this against 

6min 13s ± 19.2 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
y_pred = classifier.predict(X_test)

In [8]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[8582    0]
 [   2    4]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8582
           1       1.00      0.67      0.80         6

    accuracy                           1.00      8588
   macro avg       1.00      0.83      0.90      8588
weighted avg       1.00      1.00      1.00      8588

0.9997671169073126


In [17]:
from sklearn.externals import joblib
joblib.dump(classifier, 'classifier.pkl')
joblib.dump(X_train, 'X_train')
joblib.dump(y_train, 'y_train')
joblib.dump(X_test, 'X_test')


['X_test']

In [10]:
classifier = joblib.load('classifier.pkl')

# Oversampling 

With over sampling, there is a large issue when combining the sets. X_train is using a scipy data type. While pd.concat requires us to use data frames or series.  In order to complete these data manipulations we need to figure out how to comibe the two test sets.

Note: Do not convert X_train to a dataframe, it does not work very well

In [11]:
import scipy as sp

In [12]:
X_train= sp.sparse.csr_matrix.toarray(X_train) # In order to keep the integrity of the data this must be used before
#converting it to a data frame 

In [14]:
X_train = pd.DataFrame(X_train) #Save it as a data frame so we can upsample the data

In [19]:
X = pd.concat( [X_train,y_train ],axis =1  ) 

MemoryError: Unable to allocate array with shape (20092, 41248) and data type float64

In [ ]:
X.shape

In [ ]:
X.is_bad.value_counts()

In [ ]:
X = X[ :len(y_train)] # For some odd reason converting it to a dataframe
#gave us a length of the entire dataset rather than just the training set 
X.label = X.label.fillna(0) #In conversion, there were some lister "nan in the column"

In [ ]:
no_drugs = X[X.label ==0]
drugs = X[X.label == 1]

In [ ]:
from sklearn.utils import resample

drugs_upsampled = resample(drugs,
                          replace=True, # sample with replacement
                          n_samples=len(no_drugs), # match number in majority class
                          random_state=27) # reproducible results

In [ ]:
upsampled = pd.concat([no_drugs, drugs_upsampled])
upsampled.label.value_counts()